# WTB

Automatic classification of birds for this project : http://www.cabane-oiseaux.org/

In [20]:
import keras
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array, array_to_img
import os

DIR = '/home/kvjw3322/Documents/Prez/ML4Noobs/Notebooks/images'
image_generator = ImageDataGenerator()

categories = os.listdir(DIR)

def process_directory(path):
    print(f"Processing dir {path}")
    image_generator.flow_from_directory(path,
        target_size=(150, 150),
        batch_size=32,
        class_mode='categorical')
    
    
process_directory(DIR)


image_generator





Processing dir /home/kvjw3322/Documents/Prez/ML4Noobs/Notebooks/images
Found 3033 images belonging to 6 classes.
